# RNN 실습

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import re

from pprint import pprint

In [ ]:
sample_text = "한 사람을 구함은 세상을 구하는 것입니다. \
                당신들은 기계도 짐승도 아닙니다. 사람입니다. \
                여러분의 마음속에도 인류에 대한 사랑이 숨어서 숨쉬고 있습니다."

tf.random.set_seed(10)

In [ ]:
def preprocess_sentence(w):
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
seq_text = preprocess_sentence(sample_text)

seq_text

'<start> 한 사람을 구함은 세상을 구하는 것입니다 . 당신들은 기계도 짐승도 아닙니다 . 사람입니다 . 여러분의 마음속에도 인류에 대한 사랑이 숨어서 숨쉬고 있습니다 . <end>'

In [ ]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [ ]:
tokenize(np.array([seq_text]))

(array([[ 2,  3,  4,  5,  6,  7,  8,  1,  9, 10, 11, 12,  1, 13,  1, 14,
         15, 16, 17, 18, 19, 20, 21,  1, 22]], dtype=int32),
 <keras_preprocessing.text.Tokenizer at 0x7f96e3ad5e10>)

In [ ]:
input_tensor, inp_lang_tokenizer = tokenize(np.array([seq_text]))

for t in input_tensor[-1]:
  if t!=0:
    print ("%d ----> %s" % (t, inp_lang_tokenizer.index_word[t]))

2 ----> <start>
3 ----> 한
4 ----> 사람을
5 ----> 구함은
6 ----> 세상을
7 ----> 구하는
8 ----> 것입니다
1 ----> .
9 ----> 당신들은
10 ----> 기계도
11 ----> 짐승도
12 ----> 아닙니다
1 ----> .
13 ----> 사람입니다
1 ----> .
14 ----> 여러분의
15 ----> 마음속에도
16 ----> 인류에
17 ----> 대한
18 ----> 사랑이
19 ----> 숨어서
20 ----> 숨쉬고
21 ----> 있습니다
1 ----> .
22 ----> <end>


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

input_example, target_example = split_input_target(input_tensor[-1])

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, inp_lang_tokenizer.index_word[input_idx]))
    print("  예상 출력: {} ({:s})".format(target_idx, inp_lang_tokenizer.index_word[target_idx]))

   0단계
  입력: 2 (<start>)
  예상 출력: 3 (한)
   1단계
  입력: 3 (한)
  예상 출력: 4 (사람을)
   2단계
  입력: 4 (사람을)
  예상 출력: 5 (구함은)
   3단계
  입력: 5 (구함은)
  예상 출력: 6 (세상을)
   4단계
  입력: 6 (세상을)
  예상 출력: 7 (구하는)


In [ ]:
inp_lang_tokenizer.word_index

{'.': 1,
 '<end>': 22,
 '<start>': 2,
 '것입니다': 8,
 '구하는': 7,
 '구함은': 5,
 '기계도': 10,
 '당신들은': 9,
 '대한': 17,
 '마음속에도': 15,
 '사람을': 4,
 '사람입니다': 13,
 '사랑이': 18,
 '세상을': 6,
 '숨쉬고': 20,
 '숨어서': 19,
 '아닙니다': 12,
 '여러분의': 14,
 '인류에': 16,
 '있습니다': 21,
 '짐승도': 11,
 '한': 3}

In [ ]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(inp_lang_tokenizer.index_word) + 1

# 임베딩 차원
embedding_dim = 32

# RNN 유닛(unit) 개수
rnn_units = 256

# 배치 사이즈
batch_size = 1

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),   # faster than Dense
    tf.keras.layers.SimpleRNN(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
  ])
  return model

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

In [ ]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (1, None, 32)             736       
_________________________________________________________________
simple_rnn_18 (SimpleRNN)    (1, None, 256)            73984     
_________________________________________________________________
dense_17 (Dense)             (1, None, 23)             5911      
Total params: 80,631
Trainable params: 80,631
Non-trainable params: 0
_________________________________________________________________


### 모델 동작 확인

In [ ]:
input_x = input_example.reshape(1, -1, 1)  # batch_size, time steps, features

In [ ]:
example_batch_predictions = model(input_x)
print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

(1, 24, 23) # (배치 크기, 시퀀스 길이, 어휘 사전 크기)


In [ ]:
sampled_indices = tf.random.categorical(logits=example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

sampled_indices = np.where(sampled_indices != 0, sampled_indices, np.random.choice(np.arange(1, vocab_size)))

In [ ]:
print("입력: \n", ' '.join(map(inp_lang_tokenizer.index_word.get, input_example)))
print()
print("예측된 다음 문자: \n", ' '.join(map(inp_lang_tokenizer.index_word.get, sampled_indices)))

입력: 
 <start> 한 사람을 구함은 세상을 구하는 것입니다 . 당신들은 기계도 짐승도 아닙니다 . 사람입니다 . 여러분의 마음속에도 인류에 대한 사랑이 숨어서 숨쉬고 있습니다 .

예측된 다음 문자: 
 구함은 <start> . 구함은 숨쉬고 구하는 당신들은 아닙니다 인류에 구하는 <start> 것입니다 <end> 세상을 구하는 인류에 . 아닙니다 아닙니다 아닙니다 숨어서 사랑이 짐승도 기계도


In [ ]:
input_y = target_example.reshape(1, -1, 1)  # batch_size, time steps, features

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(input_y, example_batch_predictions)
print("스칼라 손실:          ", example_batch_loss.numpy().mean())

스칼라 손실:           3.131261


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
EPOCHS=100

history = model.fit(x=input_x, y=input_y, epochs=EPOCHS)

Epoch 1/100
1/1 [==============================] - 1s 834ms/step - loss: 3.1227
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 2.9846
Epoch 3/100
1/1 [==============================] - 0s 23ms/step - loss: 3.0568
Epoch 4/100
1/1 [==============================] - 0s 21ms/step - loss: 2.9208
Epoch 5/100
1/1 [==============================] - 0s 15ms/step - loss: 2.8740
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 2.8677
Epoch 7/100
1/1 [==============================] - 0s 14ms/step - loss: 2.8611
Epoch 8/100
1/1 [==============================] - 0s 18ms/step - loss: 2.7448
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 2.6611
Epoch 10/100
1/1 [==============================] - 0s 13ms/step - loss: 2.5853
Epoch 11/100
1/1 [==============================] - 0s 14ms/step - loss: 2.4277
Epoch 12/100
1/1 [==============================] - 0s 15ms/step - loss: 2.8494
Epoch 13/100
1/1 [==============================

![images](https://tensorflow.org/tutorials/text/images/text_generation_sampling.png)

In [ ]:
def generate_text(model, start_string, temperature):
  # 시작 문자열을 숫자로 변환(벡터화)
  input_eval = [inp_lang_tokenizer.word_index[start_string]]
  input_eval = tf.expand_dims(input_eval, 0)

  # 결과를 저장할 빈 문자열
  text_generated = []

  model.reset_states()    # 모델 초기화
  for i in range(num_generate):
    predictions = model(input_eval)

    predictions = tf.squeeze(predictions, 0)  # 배치 차원 제거
    
    # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
    # 온도가 높으면 더 의외의 텍스트가 됩니다.  
    predictions = predictions / temperature   # 온도 적용

    predicted_id = 0
    while predicted_id == 0:
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    predicted_word = inp_lang_tokenizer.index_word[predicted_id]
    text_generated.append(predicted_word)

    if predicted_word == "<end>":
      break

  return ' '.join(text_generated)

In [ ]:
start_string = '<start>'
print('temperature=1 :', generate_text(model, start_string, temperature=1))
print('temperature=0.1 :', generate_text(model, start_string, temperature=0.1))
print('temperature=0.01 :', generate_text(model, start_string, temperature=0.01))

temperature=1 : 구함은 숨어서 사람입니다 마음속에도 한 있습니다 사랑이 여러분의 구함은 구함은 당신들은 있습니다 구하는 . 짐승도 세상을 당신들은 사랑이 당신들은 <end>
temperature=0.1 : 아닙니다 기계도 구함은 세상을 구하는 . . 당신들은 기계도 짐승도 아닙니다 . 사람입니다 . 여러분의 마음속에도 인류에 대한 사랑이 숨어서 숨쉬고 있습니다 . <end>
temperature=0.01 : 한 사람을 구함은 세상을 구하는 것입니다 . 당신들은 기계도 짐승도 아닙니다 . 사람입니다 . 여러분의 마음속에도 인류에 대한 사랑이 숨어서 숨쉬고 있습니다 . <end>
